# Tutorial: Perturbation of the Neoclassical Growth Model

## Advanced Macro: numerical methods

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

In [1]:
# using ForwardDiff

In [2]:
# import ForwardDiff
# ForwardDiff.jacobian

In [7]:
using ForwardDiff: jacobian

In [12]:
function f(x)
    a = x[1]
    b = x[2] + 0.01
    for i=1:10
        b += a*i
    end
    return [(sin(a) + cos(b))^2, 1+(sin(a^2) + cos(b))]
end

f (generic function with 2 methods)

In [13]:
f([0.2, 0.4])

2-element Array{Float64,1}:
 0.36160216439462284
 1.4426536581911549

In [14]:
@time jacobian(f, [0.2, 0.4])

  0.577989 seconds (1.26 M allocations: 62.461 MiB, 3.44% gc time)


2×2 Array{Float64,2}:
 61.7259  1.10086
 50.7438  0.915348

__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__

In [8]:
using NLsolve
f(x) = [(x[1]*x[2])-1 + x[1], x[1]*x[1] + x[1] - x[2]]
x0 = [0.0, 0.0]
res = NLsolve.nlsolve(f, x0) # here there might be multiple solution depending on the initial guess

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.0, 0.0]
 * Zero: [0.5436890126920804, 0.8392867552141576]
 * Inf-norm of residuals: 0.000000
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 6
 * Jacobian Calls (df/dx): 6


Consider the deterministic neoclassical growth model

- Transition Equation
$$\begin{eqnarray}
k_t & = & (1-\delta) k_{t-1} + i_{t-1} \\\\
z_t & = & \rho z_{t-1}
\end{eqnarray}
$$

- Definition:
$$c_t = \exp(z_t) k_t^\alpha - i_t$$

- Control $i_t\in[-(1-\delta) k_t,k_t^\alpha[$
  - or equivalently $c_t \in [0, k_t^{\alpha}]$

- Objective:
$$\max_{i_t} \sum_{t\geq0} \beta^t U(c_t)$$



- Optimality Condition:
$$\beta  \left[ \frac{\left(c_{t+1}\right)^{-\gamma}}{\left(c_t\right)^{-\gamma}} \left( (1-\delta) + \alpha k_t^{\alpha -1} \right)\right] = 1$$
  - Takes into account the fact that $c_t>0$.
- Definition:
$$c_t = k_t^\alpha - i_t$$


- Calibration:
  - $\beta = 0.96$
  - $\delta = 0.1$
  - $\gamma = 4.0$
  - $\alpha = 0.3$
  - $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$
  

Our goal is to solve for the model dynamics close to the steady-state.

__Create a structure `Calibration` to hold the model parameters.__

In [9]:
struct Calibration
    α::Float64
    β::Float64
    γ::Float64
    δ::Float64
    ρ::Float64
end

In [10]:
c = Calibration(0.3, 0.96, 4.0, 0.1, 0.9)

Calibration(0.3, 0.96, 4.0, 0.1, 0.9)

In [12]:
U(x, c) = x^(1-c.γ)/(1-c.γ)

U (generic function with 1 method)

__Define two functions:__
- `transition(z, k, i, p::ModelCalibration)::Tuple{Float64, Float64}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z, k, i, Z, K, I, p::Calibration)::Tuple{Float64}` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)
- Note that we don't specify the arguments type so that we can us duck typing to perform automatic differentiation.

In [22]:
function transition(z, k, i, p::Calibration)
    Z = p.ρ*z
    K = (1-p.δ)*k + i
    return (Z, K)
end

transition (generic function with 2 methods)

In [23]:
transition(0.1, 3.0, 0.2, c)

(0.09000000000000001, 2.9000000000000004)

In [20]:
function arbitrage(z, k, i, Z, K, I, p::Calibration)
    c = exp(z)*k^p.α - i
    C = exp(Z)*K^p.α - I
    res = p.β * (C/c)^(-p.γ)*(1-p.δ + exp(Z)*k^(p.α-1)*p.α) - 1
    return [res]
end

arbitrage (generic function with 1 method)

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s, x, S, X, p::Calibration)`
- `transition(s, x, p::Calibration)`
- We aim to supply vectors, for s,x,S and X, but we don't supply arguments types: we use duck typing, in order to use automatic differentiation.

In [24]:
transition(s, x, p) = [transition(s[1], s[2], x[1], p)...]

transition (generic function with 2 methods)

In [25]:
arbitrage(s, x, S, X, p) = arbitrage(s[1],s[2], x[1], S[1], S[2], X[1], p)

arbitrage (generic function with 2 methods)

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns two vectors, one for the states, one for the controls. Check that the steady-state satisfies the model equations.


In [26]:
function steady_state(p::Calibration)
    kbar = ((1/p.β - (1-p.δ))/p.α)^(1/(p.α - 1))
    ibar = p.δ * kbar
    zbar = 0.0
    return ([zbar, kbar], [ibar])
end


steady_state (generic function with 1 method)

In [27]:
s0, x0 = steady_state(c)

([0.0, 2.920822149964071], [0.29208221499640713])

In [28]:
# this should satisfy the model equations:
arbitrage(s0, x0, s0, x0, c)

1-element Array{Float64,1}:
 0.0

In [29]:
transition(s0, x0, c) - s0

2-element Array{Float64,1}:
 0.0
 0.0

Functions `arbitrage` and `transition` corresponds to a standard representation of the model.

$$\begin{eqnarray}f(s_t , x_t, s_{t+1}, x_{t+1}) & = & 0 \\\\ 
s_{t+1} & = & g( s_t , x_t)
 \end{eqnarray}$$
 
When we differentiate it, we get:

$$\begin{eqnarray}\Delta f(s_t , x_t, s_{t+1}, x_{t+1})  =  f^{\prime}_{s_t} \Delta s_t + f^{\prime}_{x_t} \Delta x_t + f^{\prime}_{x_{t+1}} \Delta x_{t+1} +   f^{\prime}_{s_{t+1}} \Delta S_{t+1} & = & 0 \\\\
\Delta s_{t+1} & = & g^{\prime}_s \Delta s_t + g^{\prime}_x \Delta x_t\end{eqnarray}$$

where derivatives are computed at the steady-state.

The *first order* system satisfies

$$\begin{eqnarray}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{eqnarray}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



In [30]:
struct PerturbedModel
    A::Matrix{Float64}
    B::Matrix{Float64}
    C::Matrix{Float64}
    D::Matrix{Float64}
    E::Matrix{Float64}
    F::Matrix{Float64}
end

__Write a function `first_order_model(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

Partial derivatives: for a multi variable function $(x1, x2, ... x_k)\rightarrow R^n$where $_i \in R^{p_i}$(each variable is a vector), we denote by `f^{\prime}_{x_i}` the partial derivative with respect to the i-th argument. It is the derivative of function $\varphi:  R^{p_i} \rightarrow R^n, u\mapsto f(x1, \cdots, x_i,  u , x_{i+1}, \cdots) $.
Hence it is represented by a jacobian matrix $R^{p_i}\times R^n$

In [31]:
# here is how we can compute all matrices
A = jacobian((u->arbitrage(u, x0, s0, x0, c)) , s0 )
B = jacobian((u->arbitrage(s0, u, s0, x0, c)) , s0 )
C = jacobian((u->arbitrage(s0, x0, u, x0, c)) , s0 )
D = jacobian((u->arbitrage(s0, x0, s0, u, c)) , s0 )
E = jacobian((u->transition(s0, x0, c)) , s0 )
F = jacobian((u->transition(s0, x0, c)) , s0 )

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

__Define two linear operators `L_S(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__

__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blamnchard-Kahn conditions are met.__

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) s_{t-11}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots.__